In [13]:
import requests
import random
import polyline
import urllib3
import secrets
import pandas as pd
from pandas.io.json import json_normalize

In [14]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Strava API Request

In [15]:
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

print('Requesting Strava token... \n')
res = requests.post(auth_url, data=secrets.strava_payload, verify=False)
strava_access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + strava_access_token}

strava_requests_page_num = 1
all_activities = []

while True:
    strava_param = {'per_page' : 15, 'page' : strava_requests_page_num}
    strava_dataset = requests.get(activities_url, headers=header, params=strava_param).json()

    if len(strava_dataset) == 0:
        print('breaking out of Strava while loop because the response is zero, indicating no more activities.')
        break

    if all_activities:
        print('all activities is populated')
        all_activities.extend(strava_dataset)

    else:
        print('all activities is NOT populated')
        all_activities = strava_dataset

    strava_requests_page_num += 1

print('Total Activities: ', len(all_activities))

Requesting Strava token... 



all activities is NOT populated
all activities is populated
all activities is populated
all activities is populated
breaking out of Strava while loop because the response is zero, indicating no more activities.
Total Activities:  49


In [16]:
run_data = pd.DataFrame(data=all_activities)

In [17]:
run_data.columns

Index(['resource_state', 'athlete', 'name', 'distance', 'moving_time',
       'elapsed_time', 'total_elevation_gain', 'type', 'sport_type',
       'workout_type', 'id', 'start_date', 'start_date_local', 'timezone',
       'utc_offset', 'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'map', 'trainer', 'commute', 'manual', 'private',
       'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng',
       'average_speed', 'max_speed', 'has_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'suffer_score', 'average_watts',
       'kilojoules', 'device_watts'],
      dtype='object')

In [18]:
run_data.head()

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_watts,kilojoules,device_watts
0,2,"{'id': 8586088, 'resource_state': 1}",Ten Sleep Morning Run,2927.4,937,943,27.1,Run,Run,0.0,...,9970213520,D9326B4B-A20A-4FFC-963A-A75D2D17971D-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
1,2,"{'id': 8586088, 'resource_state': 1}",Ten Sleep Canyon Old Road Morning Run,3801.4,1185,1327,61.0,Run,Run,0.0,...,9970213502,9E9E8812-BB6B-4016-96DA-8E72047B2967-activity.fit,False,2,0,False,NaN,NaN,NaN,NaN
2,2,"{'id': 8586088, 'resource_state': 1}",Grays Arch loop w/ Rush,6222.5,2225,2271,173.6,Run,Run,0.0,...,9845666953,5AC4EBC8-8F82-40A1-A71E-464C999BD249-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
3,2,"{'id': 8586088, 'resource_state': 1}",PMRP Rain Run,8973.2,2836,2882,368.6,Run,Run,0.0,...,9827119565,8FDC9AE7-669F-463F-8FE7-69ADAECECDF6-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
4,2,"{'id': 8586088, 'resource_state': 1}",Lot - Lode clockwise - Lot - Lode Hill up to p...,2615.8,843,849,96.5,Run,Run,0.0,...,9719069921,24A57C94-9908-447D-A155-9BDE2019A7B9-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN


In [19]:
# map_data = json_normalize(run_data['map'])

# run_data_map = pd.DataFrame(map_data)

# run_data = pd.merge(run_data, run_data_map, on='resource_state', how='inner')

columns_to_drop = ['athlete', 
                   'sport_type', 
                   'workout_type', 
                   'kudos_count', 
                   'comment_count', 
                   'athlete_count', 
                   'photo_count', 
                   'trainer', 
                   'commute', 
                   'manual', 
                   'private',
                   'visibility', 
                   'flagged', 
                   'gear_id', 
                   'has_heartrate', 
                   'heartrate_opt_out', 
                   'display_hide_heartrate_option', 
                   'from_accepted_tag', 
                   'total_photo_count', 
                   'has_kudoed', 
                   'average_watts', 
                   'kilojoules', 
                   'device_watts', 
                   'suffer_score']
run_data.drop(columns=columns_to_drop, inplace=True)

In [20]:
run_data.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'id', 'start_date', 'start_date_local',
       'timezone', 'utc_offset', 'location_city', 'location_state',
       'location_country', 'achievement_count', 'map', 'start_latlng',
       'end_latlng', 'average_speed', 'max_speed', 'elev_high', 'elev_low',
       'upload_id', 'upload_id_str', 'external_id', 'pr_count'],
      dtype='object')

## Conversions for Metrics

In [21]:
run_data['distance_miles'] = run_data['distance'] * 0.00062137119
run_data['moving_time_minutes'] = run_data['moving_time'] / 60
run_data['moving_time_hours'] = run_data['moving_time'] / 3600


In [22]:
run_data

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,id,start_date,start_date_local,...,max_speed,elev_high,elev_low,upload_id,upload_id_str,external_id,pr_count,distance_miles,moving_time_minutes,moving_time_hours
0,2,Ten Sleep Morning Run,2927.4,937,943,27.1,Run,9294421659,2023-06-19T12:34:47Z,2023-06-19T06:34:47Z,...,4.750,1655.9,1583.3,9970213520,9970213520,D9326B4B-A20A-4FFC-963A-A75D2D17971D-activity.fit,0,1.819002,15.616667,0.260278
1,2,Ten Sleep Canyon Old Road Morning Run,3801.4,1185,1327,61.0,Run,9294421645,2023-06-17T12:57:06Z,2023-06-17T06:57:06Z,...,6.196,1645.2,1583.4,9970213502,9970213502,9E9E8812-BB6B-4016-96DA-8E72047B2967-activity.fit,2,2.362080,19.750000,0.329167
2,2,Grays Arch loop w/ Rush,6222.5,2225,2271,173.6,Run,9176575201,2023-05-31T15:42:37Z,2023-05-31T11:42:37Z,...,7.315,392.0,255.0,9845666953,9845666953,5AC4EBC8-8F82-40A1-A71E-464C999BD249-activity.fit,0,3.866482,37.083333,0.618056
3,2,PMRP Rain Run,8973.2,2836,2882,368.6,Run,9159069458,2023-05-28T18:17:37Z,2023-05-28T14:17:37Z,...,16.328,349.9,259.0,9827119565,9827119565,8FDC9AE7-669F-463F-8FE7-69ADAECECDF6-activity.fit,0,5.575688,47.266667,0.787778
4,2,Lot - Lode clockwise - Lot - Lode Hill up to p...,2615.8,843,849,96.5,Run,9059541871,2023-05-12T19:53:27Z,2023-05-12T15:53:27Z,...,7.168,339.3,267.6,9719069921,9719069921,24A57C94-9908-447D-A155-9BDE2019A7B9-activity.fit,0,1.625383,14.050000,0.234167
5,2,First Blue Loop Run - Lode lot - Velo - bushwh...,9653.3,3349,3385,453.1,Run,9017048707,2023-05-05T18:39:29Z,2023-05-05T14:39:29Z,...,14.712,332.0,259.2,9673999538,9673999538,A1D712A9-AE60-46D6-BC79-079F26074906-activity.fit,2,5.998283,55.816667,0.930278
6,2,Redpoint Disc Golf Course,4160.1,3589,4261,140.4,Walk,8985168804,2023-04-30T14:19:18Z,2023-04-30T10:19:18Z,...,3.952,362.3,317.1,9640254989,9640254989,3D979EEF-835D-4FFD-8A78-C3D597E4ACD6-activity.fit,0,2.584966,59.816667,0.996944
7,2,PMRP Red Loop (drive by-bob-lot-tow-gmc-lot),5774.7,1769,1827,131.6,Run,8973585450,2023-04-28T18:23:12Z,2023-04-28T14:23:12Z,...,11.087,333.2,258.4,9627942193,9627942193,2F517B9E-B515-40EF-8768-5870F570A507-activity.fit,2,3.588232,29.483333,0.491389
8,2,Beer trailer unsuccessful morel walk,2995.8,1603,1996,180.9,Walk,8903223949,2023-04-16T17:47:53Z,2023-04-16T13:47:53Z,...,12.074,351.9,292.5,9553234181,9553234181,EC96B862-45EA-45DA-970E-3F3E763408AF-activity.fit,0,1.861504,26.716667,0.445278
9,2,PMRP run before work,6867.9,2329,2376,228.5,Run,8885448119,2023-04-13T18:21:31Z,2023-04-13T14:21:31Z,...,13.132,333.6,259.0,9534311976,9534311976,89B2BBFD-A972-4F20-97ED-AC0A0EBA02C3-activity.fit,0,4.267515,38.816667,0.646944


In [23]:
average_distance_miles = run_data['distance_miles'].mean()
print("Average Distance (miles):", average_distance_miles)

average_time_minutes = run_data['moving_time_minutes'].mean()
print("Average Time Ran (minutes):", average_time_minutes)

max_distance_ran = run_data['distance_miles'].max()
print("Longest Run:", max_distance_ran, "miles")

max_duration_mintues = run_data['moving_time_minutes'].max()
max_duration_hours = run_data['moving_time_hours'].max()
print("Longest Duration:", max_duration_mintues,"minutes. Converted to hours:", max_duration_hours)

total_distance_miles = run_data['distance'].sum() * 0.00062137119
print("Total Distance Covered to the date (miles):", total_distance_miles)

Average Distance (miles): 3.402851822826613
Average Time Ran (minutes): 40.167006802721076
Longest Run: 9.618018238653 miles
Longest Duration: 117.56666666666666 minutes. Converted to hours: 1.9594444444444445
Total Distance Covered to the date (miles): 166.73973931850398
